# Simple Example
This Jupyter notebook runs on Colab and shows a simple example of Tooling selection using Top-K and Double Round Robin

## Install Ollama

Before we get started with Mellea, we download, install and serve ollama. We define set_css to wrap Colab output.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

from IPython.display import HTML, display


def set_css():
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))


get_ipython().events.register("pre_run_cell", set_css)

## Install Mellea
We run `uv pip install mellea` to install Mellea.

In [ ]:
!uv pip install mellea -q

### **Import top_k and double_round_robin libraries**

In [ ]:
!git clone https://github.com/generative-computing/mellea-contribs.git

In [ ]:
%cd mellea-contribs
!pip install -e . -qq

## **Defining sample set of tools**

In [ ]:
from mellea import start_session
import mellea

# Start a Mellea session
m = start_session()

# Define a sample set of tools
TOOLS = [
    {
        "name": "EnterpriseScheduler",
        "short_description": "Schedules internal meetings and events.",
        "long_description": (
            "EnterpriseScheduler is an internal tool used to schedule meetings and events across teams. "
            "It integrates with company calendars, room booking systems, and employee availability data "
            "to resolve conflicts and enforce scheduling policies. The tool focuses strictly on time "
            "and resource coordination and does not perform financial analysis or budgeting."
        ),
        "requirements": {
            "accepts": ["date", "participants", "location"],
            "does_not_support": ["budget_calculation", "cost_estimation"]
        }
    },
    {
        "name": "BudgetOptimizer",
        "short_description": "Performs internal budget forecasting and cost analysis.",
        "long_description": (
            "BudgetOptimizer is a proprietary financial planning tool used for estimating costs, "
            "forecasting budgets, and optimizing spend for internal initiatives. It integrates with "
            "enterprise ERP systems and historical financial data to produce projections and summaries. "
            "The tool does not schedule meetings or manage calendars."
        ),
        "requirements": {
            "accepts": ["cost_inputs", "financial_constraints"],
            "does_not_support": ["meeting_scheduling", "calendar_management"]
        }
    },
    {
        "name": "DocSearchPro",
        "short_description": "Searches internal documents and knowledge bases.",
        "long_description": (
            "DocSearchPro provides semantic search over internal documents, policies, and project files. "
            "It is designed to help employees retrieve institutional knowledge and past decisions quickly. "
            "The tool does not execute actions, perform calculations, or modify schedules."
        ),
        "requirements": {
            "accepts": ["search_query", "filters"],
            "does_not_support": ["scheduling", "budgeting", "resource_allocation"]
        }
    },
    {
        "name": "ResourceAllocator",
        "short_description": "Allocates people and resources to projects.",
        "long_description": (
            "ResourceAllocator manages internal project resources by assigning people, equipment, and "
            "timelines based on availability and priority constraints. It integrates with HR and project "
            "management systems to track utilization and resolve conflicts. The tool does not handle "
            "meeting scheduling or financial budgeting."
        ),
        "requirements": {
            "accepts": ["project_id", "resource_constraints"],
            "does_not_support": ["meeting_scheduling", "budget_calculation"]
        }
    }
]

USER_QUERY = ("Plan a team kickoff for next week by scheduling a meeting and estimating the expected cost based on headcount and duration.")

## **Step 1: Top-K Tool Shortlisting**

In [ ]:
from mellea_contribs.tools.top_k import top_k
from pydantic import RootModel
from mellea.stdlib.requirement import check, req, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

def shortlist_tools(m, user_query, tools, k):
    items = [
        {"name": t["name"], "description": t["short_description"]}
        for t in tools
    ]

    ranked = top_k(
        items=items,
        comparison_prompt=f"""
        Select the tools most relevant to:
        '{user_query}'
        """,
        m=m,
        k=k,
    )

    ranked_names = {t["name"] for t in ranked}
    return [t for t in tools if t["name"] in ranked_names]

## **Step 2: High-Confidence Ranking with Double Round Robin**

In [ ]:
from mellea_contribs.tools.double_round_robin import double_round_robin

def rank_tools(m, user_query, tools):
    items = [
        {
            "name": t["name"],
            "description": t["long_description"],
            "constraints": t["requirements"],
        }
        for t in tools
    ]

    return double_round_robin(
        items=items,
        comparison_prompt=f"""
        Given the user query:
        '{user_query}'

        Compare tools strictly based on their described capabilities
        and stated constraints. Do not assume unsupported functionality.
        """,
        m=m,
    )


## **Final Tool selection**

In [ ]:
class ToolChoice(RootModel[str]):
    pass

def choose_final_tool(
    m: mellea.MelleaSession,
    user_query: str,
    ranked_tools: list[dict],
):
    tool_names = [t["name"] for t in ranked_tools]

    tool_context = "\n\n".join(
        f"""
        Tool: {t['name']}
        Description: {t['long_description']}
        Requirements: {t['requirements']}
        """
        for t in ranked_tools
    )

    response = m.instruct(
        f"""
        Given the user query:
        '{user_query}'

        Choose the single best tool.
        The tool must satisfy the user's intent and must not violate its stated requirements.
        """,
        grounding_context={
          "tools": tool_context
        },
        requirements=[
            req(
                "Tool must be one of the provided candidates",
                validation_fn=simple_validate(lambda s: s in tool_names),
            ),
            req("Return only the tool name"),
        ],
        strategy=RejectionSamplingStrategy(loop_budget=3),
        format=ToolChoice,
    )

    return response.value


In [ ]:
m = start_session()

top_k_tools = shortlist_tools(m, USER_QUERY, TOOLS, k=2)
ranked = rank_tools(m, USER_QUERY, top_k_tools)

ranked_tools = []
tool_by_name = {t["name"]: t for t in TOOLS}

for item, score in ranked:
    ranked_tools.append(tool_by_name[item["name"]])

final_tool = choose_final_tool(m, USER_QUERY, ranked_tools)

print("Top-K shortlisted tools:", [t["name"] for t in top_k_tools])
print("DRR ranking:", [(t["name"], score) for t, score in ranked])
print("Final selected tool:", final_tool)